# LZW [[Welch, 1984]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Terry+Welch+1984&btnG=)

* In 1984, Terry A. Welch proposed the [LZW algorithm](https://en.wikipedia.org/wiki/Lempel%E2%80%93Ziv%E2%80%93Welch),
  an improved version of the LZ78 algorithm that does not
  writes raw symbols ($k$ fields) to the code-stream.

* LZW was selected as the encoding engine for the [GIF (Graphics
  Interchange Format)](https://en.wikipedia.org/wiki/GIF), and for the compressor [`compress`](https://en.wikipedia.org/wiki/Compress).
  
* Initially, the dictionary is filled with the $2^k$ possible
  symbols (*roots*), that are stored in the first entries (for 1-byte symbols: $0\cdots255$).

## Encoder

1. $w\leftarrow$ first input symbol.
2. While the input is not exhausted:
    1. $k\leftarrow$ next input symbol.
    2. If $wk$ is found in the dictionary, then:
        1. $w\leftarrow$ address of $wk$ in the dictionary.
    3. Else:
        1. Output $\leftarrow w$.
        2. Insert $wk$ in the dictionary.
        3. $w\leftarrow k$.

## Example
<img src="data/LZW_encoding_example.png" width="400">

## Decoder

1. $c\leftarrow$ first input code-word.
2. Output $c$.
3. $c'\leftarrow c$.
4. While the input is not exhausted:
    1. $c\leftarrow$ next input code-word.
    2. $w\leftarrow c'$.
    3. If $c$ is found in the dictionary, then:
        1. Output string$(c)$.
    4. Else:
        1. Output string$(w)$.
        2. Output $k$.
    5. $k\leftarrow$ first symbol of the last output.
    6. Insert $wk$ in the dictionary.
    7. $c'\leftarrow c$.

## Example
<img src="data/LZW_decoding_example.png" width="600">

## Lab

In [4]:
# https://rosettacode.org/wiki/LZW_compression#Python

# TO-DO
import io

def encoder(uncompressed):
    """Compress a string to a list of output symbols."""
 
    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}
 
    w = ""
    result = []
    for k in uncompressed:
        wk = w + k
        if wk in dictionary:
            w = wk
        else:
            result.append(dictionary[w])
            # Add wk to the dictionary.
            dictionary[wk] = dict_size
            dict_size += 1
            w = k
 
    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result
 
 
   
def decoder(compressed):
    """Decompress a list of output ks to a string."""
     
    # Build the dictionary.
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}
 
    # use io.StringIO(), otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = io.StringIO()
    w = chr(compressed.pop(0))
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result.write(entry)
 
        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1
 
        w = entry
    return result.getvalue()
 

# Testing the algorithm
cadena = "Hola, soy Claudio y este algoritmo es correcto..."
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

# Other examples
cadena = "AABBAAABBBAAAABBBBAAAAABBBBBABABA"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

cadena = "TOBEORNOTTOBEORTOBEORNOT"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

[72, 111, 108, 97, 44, 32, 115, 111, 121, 32, 67, 258, 117, 100, 105, 111, 32, 264, 101, 115, 116, 101, 32, 97, 108, 103, 111, 114, 105, 116, 109, 271, 274, 32, 99, 282, 114, 101, 99, 116, 111, 46, 297]
Hola, soy Claudio y este algoritmo es correcto...

[65, 65, 66, 66, 256, 257, 258, 260, 261, 262, 263, 258, 267, 257, 269]
AABBAAABBBAAAABBBBAAAAABBBBBABABA

[84, 79, 66, 69, 79, 82, 78, 79, 84, 256, 258, 260, 265, 259, 261, 263]
TOBEORNOTTOBEORTOBEORNOT
